In [2]:
!pip install hyperopt xgboost mlflow boto3 -q


This code sets up the hyperparameter tuning process using hyperopt, logs the parameters and metrics to MLflow, and runs the optimization. After running the optimization, you will be able to see the results in the MLflow UI.

### 1: Import Libraries and Suppress Warnings


In [6]:
import mlflow
import mlflow.xgboost
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import warnings
from sklearn.preprocessing import StandardScaler

# Suppress specific warnings
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost')
warnings.filterwarnings('ignore', category=UserWarning, module='_distutils_hack')

In [8]:

# Set MLflow tracking URI
EC2_PUBLIC_DNS='ec2-16-16-217-131.eu-north-1.compute.amazonaws.com'
mlflow.set_tracking_uri(f"http://{EC2_PUBLIC_DNS}:5000")
mlflow.set_experiment("xgboost_hyperparameter_tuning")


<Experiment: artifact_location='s3://mlflow-artifacts-capstone-mlops/4', creation_time=1722115105130, experiment_id='4', last_update_time=1722115105130, lifecycle_stage='active', name='xgboost_hyperparameter_tuning', tags={}>

### 2: Load Data and Preprocess


In [2]:
import pandas as pd

# Load the cleaned data
cleaned_df = pd.read_csv('../../data/cleaned_car_data.csv')
cleaned_df.head()

,year,price,mileage,enginesize,tax,mpg,make_bmw,make_cclass,make_focus,make_ford,make_hyundi,make_merc,make_skoda,make_toyota,make_vauxhall,make_vw,transmission_Manual,transmission_Semi-Auto,fueltype_Hybrid,fueltype_Petrol
0,2016.0,8000.0,38852.0,1.0,145.0,54.3,False,False,True,False,False,False,False,False,False,False,True,False,False,True
1,2019.0,13400.0,11952.0,1.0,145.0,54.3,False,False,True,False,False,False,False,False,False,False,True,False,False,True
2,2019.0,14600.0,22142.0,1.5,145.0,54.3,False,False,True,False,False,False,False,False,False,False,True,False,False,True
3,2016.0,9450.0,14549.0,1.6,145.0,54.3,False,False,True,False,False,False,False,False,False,False,True,False,False,False
4,2015.0,9999.0,7010.0,1.6,145.0,54.3,False,False,True,False,False,False,False,False,False,False,True,False,False,False


In [7]:
# Load the cleaned data
cleaned_df = pd.read_csv('../../data/cleaned_car_data.csv')

# Define features (X) and target variable (y)
X = cleaned_df.drop(columns=['price'])  # Exclude 'price'
y = cleaned_df['price']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shape of the splits
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

# Optional: Check the first few rows of the scaled data
print("First row of the training data:")
print(X_train[:1])


Training data shape: (84888, 19)
Testing data shape: (21223, 19)
First row of the training data:
         year  mileage  enginesize    tax   mpg  make_bmw  make_cclass   
71413  2020.0   4750.0         1.5  145.0  44.8     False        False  \

       make_focus  make_ford  make_hyundi  make_merc  make_skoda  make_toyota   
71413       False      False        False      False       False        False  \

       make_vauxhall  make_vw  transmission_Manual  transmission_Semi-Auto   
71413          False    False                 True                   False  \

       fueltype_Hybrid  fueltype_Petrol  
71413            False             True  


### 3: Define Hyperparameter Space and Objective Function


In [10]:
space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 100, 1000, 100),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'seed': 42,
    'objective': 'reg:squarederror'
}

def objective(params):
    with mlflow.start_run(nested=True):
        # Convert hyperparameters to int if they are passed as floats
        params["max_depth"] = int(params["max_depth"])
        params["n_estimators"] = int(params["n_estimators"])

        # Log hyperparameters
        mlflow.log_params(params)

        # Train the model
        model = xgb.XGBRegressor(
            n_estimators=params["n_estimators"],
            max_depth=params["max_depth"],
            learning_rate=params["learning_rate"],
            subsample=params["subsample"],
            seed=params["seed"],
            objective=params["objective"]
        )
        model.fit(X_train, y_train)

        # Predict and evaluate
        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

        # Log the model
        mlflow.xgboost.log_model(model, "model")

        return {'loss': rmse, 'status': STATUS_OK}


### 4: Run Hyperparameter Optimization


In [11]:
trials = Trials()
best_params = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=50,  # Adjust this number based on how long you want to search
    trials=trials
)

print()
print()

print("Best parameters:", best_params)


100%|██████████| 50/50 [07:44<00:00,  9.30s/trial, best loss: 2285.035402316803]


Best parameters: {'learning_rate': 0.09455111298980684, 'max_depth': 9.0, 'min_child_weight': 0.3730492049381335, 'n_estimators': 500.0, 'subsample': 0.9989341273723211}
